In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt

In [ ]:
torch.manual_seed(3)
BATCH_SIZE = 64
NUMBER_OF_EPOCH = 100

device = torch.device("cuda")

kwargs = {'num_workers': 1, 'pin_memory': True} 
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('/./FashionMNIST', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('/./FashionMNIST', train=False, transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc2 = nn.Linear(400, 400)
        self.fc31 = nn.Linear(400, 20)
        self.fc32 = nn.Linear(400, 20)
        self.fc4 = nn.Linear(20, 400)
        self.fc5 = nn.Linear(400, 400)
        self.fc6 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        h1 = F.relu(self.fc2(h1))
        return self.fc31(h1), self.fc32(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc4(z))
        h3 = F.relu(self.fc5(h3))
        return torch.sigmoid(self.fc6(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [ ]:
# Reconstruction + KL divergence losses summed over all elements and batch
# https://arxiv.org/abs/1312.6114
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [ ]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        LOG_INTERVALS = 300
        # if batch_idx % LOG_INTERVALS == 0:
        #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #         epoch, batch_idx * len(data), len(train_loader.dataset),
        #         100. * batch_idx / len(train_loader),
        #         loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [ ]:
losses = []
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n], recon_batch.view(BATCH_SIZE, 1, 28, 28)[:n]])
                save_image(comparison.cpu(), f'./reconstruction_{epoch}.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    losses.append(test_loss)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
print(device)
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

cuda


In [ ]:
torch.manual_seed(45)
for epoch in range(1, NUMBER_OF_EPOCH+1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, 20).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28), f'./sample_{epoch}.png')
        

====> Epoch: 1 Average loss: 276.0783
====> Test set loss: 255.7206
====> Epoch: 2 Average loss: 250.9982
====> Test set loss: 250.2790
====> Epoch: 3 Average loss: 247.1881
====> Test set loss: 247.6669
====> Epoch: 4 Average loss: 245.1567
====> Test set loss: 246.2467
====> Epoch: 5 Average loss: 243.6794
====> Test set loss: 244.5136
====> Epoch: 6 Average loss: 242.5068
====> Test set loss: 244.8574
====> Epoch: 7 Average loss: 241.6922
====> Test set loss: 243.7591
====> Epoch: 8 Average loss: 241.0743
====> Test set loss: 242.9837
====> Epoch: 9 Average loss: 240.5762
====> Test set loss: 242.2145
====> Epoch: 10 Average loss: 240.1740
====> Test set loss: 241.8867
====> Epoch: 11 Average loss: 239.8287
====> Test set loss: 241.5618
====> Epoch: 12 Average loss: 239.5613
====> Test set loss: 241.5181
====> Epoch: 13 Average loss: 239.3318
====> Test set loss: 241.6806
====> Epoch: 14 Average loss: 239.1182
====> Test set loss: 240.7976
====> Epoch: 15 Average loss: 238.9132
====

In [ ]:
!zip -r "/content/img.zip" "/content/"

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2023.04.14/ (stored 0%)
  adding: content/.config/logs/2023.04.14/13.33.49.775521.log (deflated 57%)
  adding: content/.config/logs/2023.04.14/13.33.23.944257.log (deflated 91%)
  adding: content/.config/logs/2023.04.14/13.34.46.037681.log (deflated 56%)
  adding: content/.config/logs/2023.04.14/13.34.12.944578.log (deflated 86%)
  adding: content/.config/logs/2023.04.14/13.34.45.328952.log (deflated 57%)
  adding: content/.config/logs/2023.04.14/13.34.21.160489.log (deflated 58%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 23%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/co